In [1]:
import logging
import os
import sys

from cookbook.interface import PluginInstanceRedisInterface
from rmsd_v2.fpocket_client import FPocketClient


logging.basicConfig()
logger = logging.getLogger(__name__)

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''
redis_channel = os.environ.get("REDIS_CHANNEL")

# Increase the recursion limit in order to properly serialize Complexes
recursion_limit = 100000
sys.setrecursionlimit(recursion_limit)

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
fpocket_client = FPocketClient()

In [ ]:
# Load in example structures

from nanome.api.structure import Complex
from nanome.util import Color
# Load Test Molecules
examples_folder = os.path.join(os.getcwd(), 'examples')
pdb_6nzt = os.path.join(examples_folder, '6nzt.pdb')
pdb_6nzv = os.path.join(examples_folder, '6nzv.pdb')

comp_1 = Complex.io.from_pdb(path=pdb_6nzt)
comp_1.name = "6NZT"
comp_2 = Complex.io.from_pdb(path=pdb_6nzv)
comp_2.name = "6NZV"
comp_list = [comp_1, comp_2]
moving_comp = next(iter([comp in comp_list if comp.name == '6NZT']))
# fixed_comp = next(comp in comp_list if comp.name == '6NZV')

# plugin_instance.update_structures_deep(comp_list)


In [16]:
ws = plugin_instance.request_workspace()
comp_list = ws.complexes

moving_comp = next(comp for comp in comp_list if comp.name == '6NZT')
fixed_comp = next(comp for comp in comp_list if comp.name == '6NZV')

Sending request_workspace Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel a9e1b46f-eff0-4db7-9bd8-3a2c9fca97c4


In [21]:
# Extract the binding pocket surrounding the ligand.
import copy
from rmsd_v2.RMSDV2 import RMSDV2
from nanome.api import structure
from nanome.util import Color
instance = RMSDV2()

comp = fixed_comp
ligand_name = 'L9J'


def extract_binding_site(comp, binding_site_atoms):
    # Copy comp, and remove all atoms that are not part of the binding site
    new_comp = copy.deepcopy(comp)
    new_comp.name = f'{ligand_name} binding site'
    new_comp.index = -1
    new_comp.set_surface_needs_redraw()
    
    binding_site_indices = [a.index for a in binding_site_atoms]
    print(len(binding_site_indices))
    for mol in new_comp.molecules:
        mol.index = -1
        for chain in mol.chains:
            if sum(1 for a in chain.atoms if a.index in binding_site_indices) == 0:
                mol.remove_chain(chain)
                continue
            chain.index = -1
            for residue in chain.residues:
                if sum(1 for a in residue.atoms if a.index in binding_site_indices) == 0:
                    chain.remove_residue(residue)
                    continue
                residue.index = -1
                for atom in residue.atoms:
                    atom.index = -1
                    if atom.index not in binding_site_indices:
                        atom.residue.remove_atom(atom)
                    else:
                        atom.index = -1
    return new_comp

def get_binding_site_atoms(comp, ligand_name):
    residue = next(res for res in comp.residues if res.name == ligand_name)
    binding_site_atoms = instance.calculate_binding_site_atoms(comp, residue.atoms)
    return binding_site_atoms

def highlight_atoms(atom_list, on_or_off: bool):
    for atom in atom_list:
        atom.selected = on_or_off
    plugin_instance.update_structures_deep([comp])

fixed_binding_site_atoms = get_binding_site_atoms(fixed_comp, ligand_name)
print(f'{len(fixed_binding_site_atoms)} Atoms in binding_site')
fixed_binding_site = extract_binding_site(fixed_comp, fixed_binding_site_atoms)

print(f'{len(list(fixed_binding_site.atoms))} Atoms in extracted binding_site')
highlight_atoms(fixed_binding_site_atoms, True)
plugin_instance.update_structures_deep([fixed_binding_site])
# highlight_atoms(fixed_binding_site_atoms, True)



84 Atoms in binding_site
84
689 Atoms in extracted binding_site
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 355725f4-ae37-4220-8165-809ccadfc358
Sending update_structures_deep Request to Redis Channel 78bdbce1-5f84-44ec-ae08-79f2768351b5
Response received on channel 516c2015-fcc9-4187-847d-2fdac5e82158


()

In [ ]:
# Run FPocket, and highlight the moving structure potential pockets

import time

comp = moving_comp
ligand_name = 'L9P'
    
def cycle_pockets(comp):
    fpocket_client = FPocketClient()
    logger.info(f"Showing pockets for {comp.name}")
    pocket_atoms = fpocket_client.get_pockets_atoms(comp)
    for pocket in pocket_sets:
        print(f"Highlighting pocket {current_index}")
        highlight_atoms(comp, pocket)

    # Deselect all atoms
    for atom in comp.atoms:
        atom.selected = False
    plugin_instance.update_structures_deep([comp])


# Run Fpocket to get potential pockets on moving complex, and find best match to fixed complex using SiteMotif
output_dir = f'{moving_comp.name}_out'
fpocket_results = fpocket_client.run(moving_comp, output_dir)
pocket_folder = os.path.join(fpocket_results, 'pockets')
moving_pocket_pdbs = sorted([f'{pocket_folder}/{fi}' for fi in os.listdir(pocket_folder) if fi.endswith('.pdb')])

# binding_site_atoms = plugin_instance.get_binding_site_atoms(comp, ligand_name)
# cycle_pockets(comp)


In [ ]:
import subprocess

multi_exe_path = '/app/cookbook/site_motif/MulitpleSiteAlignment/pocket_matrix_mpi7.py'
exe_path = '/app/cookbook/site_motif/PairWiseComparison/pocket_matrix7.py'

fixed_binding_site = '/app/cookbook/notebooks/L9P_binding_site.pdb'
moving_binding_site = '/app/cookbook/notebooks/pocket1_atm.pdb'

cmd = f'conda run -n py27 python {exe_path} {moving_binding_site} {fixed_binding_site}' 
subprocess.run(cmd.split())


In [ ]:
cmd.split()